In [1]:
import sys
import pickle
from scipy import signal
from scipy import stats
import numpy as np
from sklearn.model_selection import ShuffleSplit
import math
from collections import OrderedDict


import matplotlib.pyplot as plt

sys.path.append('D:\Diamond\code')
from csp_james_2 import *

sys.path.append('D:\Diamond\code')
from thesis_funcs_19_03 import *

import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler

import csv
import datetime


In [31]:
k_fold = 10

FS= [250]
num_epoch = 700



meth = 'tl_comp_csp_mi' #gold_stand，tl_comp_csp_kld , tl_comp_csp_mi
config_root= 'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\configs\\'
feature_root = 'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\CURRENT\\' + meth + '\\'
model_root = feature_root
save_root = model_root


#load in cv config grid
hp_names  =[] #all the hyper-parameter names to be validated
with open(config_root +'cv_config.csv', mode = 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    for row in csv_reader:
        hp_names.append((row[0]).strip())

with open(config_root +'_lambda_config.csv', mode = 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    for row in csv_reader:
        hp_names.append((row[0]).strip())
csv_file.close()

In [32]:
to_save = 1

In [34]:
num_inits = 5

for subject in range (1,10):

    filename = 'A0'+str(subject)+'T'
    filename_save = filename
    print (filename_save)
    
    for portion_train in [1,0.7,0.5,0.3,0.1]:
        print (portion_train)
        
        for run_win in range (0,1):
            if run_win == 0:
                file_root_feature =  feature_root + filename_save[:-1] + '\\4s\\' + 'pt_' + str(int(portion_train*100))
                file_root_model = model_root + filename_save[:-1] + '\\4s\\' + 'pt_' + str(int(portion_train*100))
                file_root_save = save_root + filename_save[:-1] + '\\4s\\' + 'pt_' + str(int(portion_train*100))
                #len_inp = 44
            elif run_win == 1:
                file_root_feature = feature_root + filename_save[:-1] + '\\2s\\' + 'pt_' + str(int(portion_train*100))
                file_root_model = model_root + filename_save[:-1] + '\\2s\\' + 'pt_' + str(int(portion_train*100))
                file_root_save = save_root + filename_save[:-1] + '\\2s\\' + 'pt_' + str(int(portion_train*100))
                #len_inp = 25
                
            
            LABELS0_go = pickle.load(open(file_root_feature + '\\LABELS0_go.pickle', 'rb'))
            TRAIN_IDX = pickle.load(open(file_root_feature  + '\\TRAIN_IDX.pickle', 'rb'))
            TEST_IDX = pickle.load(open(file_root_feature + '\\TEST_IDX.pickle', 'rb'))  
            
            batch_size = np.shape(TRAIN_IDX)[-1]
            #batch_size = 50
            
            ###################################################################################################################
                            #load best config
            ###################################################################################################################
            #load in best config line
            config_file = open(file_root_model + '\\ANN\\best_config_val.txt', 'r')
            config_log= config_file.readlines()
            config_file.close()
            for i in range (0,len(config_log)):
                line  = config_log[(i + 1) * -1]
                if '_act_fun_' in line: #and  line.split(' ')[0].split('_lambda_')[1] == '0':
                    break

            #extract best config values and make into dictionary
            config = OrderedDict()
            for hp_ind in range(0, len(hp_names)-1):
                config[hp_names[hp_ind]] =  (line.split(hp_names[hp_ind] + '_')[1].split('_'+hp_names[hp_ind+1]+'_')[0])
            config[hp_names[-1]] = line.split(hp_names[-1]+'_')[1].split(' ')[0]
            
            ACC = []
            for n_inits in range (0, num_inits):
                #print (n_inits)
                ###############################################################################################################################
                for fold in range (0, k_fold):
                    #print (fold)
                    train_idx0 = TRAIN_IDX[fold]
                    test_idx0 = TEST_IDX[fold]

                    trn_ind_map = np.arange(0,len(train_idx0))
                    tst_ind_map = np.arange(0,len(test_idx0))

                    random.shuffle(trn_ind_map)
                    random.shuffle(tst_ind_map)

                    y_train = LABELS0_go[train_idx0[trn_ind_map]]
                    y_test = LABELS0_go[test_idx0[tst_ind_map]]
                    
                    X_train0 = pickle.load(open(file_root_feature + '\\Z_all_classes_train_fold_' + str(fold) + 
                                                                '_lambda_' + str(float(config['_lambda'].strip())) + ".pickle", 'rb'))

                    X_test0 = pickle.load(open( file_root_feature + '\\Z_all_classes_test_fold_' + str(fold) + 
                                               '_lambda_' + str(float(config['_lambda'].strip())) + ".pickle", 'rb'))
                    
                    X_train0 = X_train0[trn_ind_map]
                    X_test0 = X_test0[tst_ind_map]

                    #reshape to fit 2d covolution ( 1d doesnt work)
                    X_train = np.reshape(X_train0, [np.shape(X_train0)[0], 1, np.shape(X_train0)[1], np.shape(X_train0)[2]]).astype('float64')
                    X_test = np.reshape(X_test0, [np.shape(X_test0)[0], 1, np.shape(X_test0)[1], np.shape(X_test0)[2]]).astype('float64')

                    #convert to torch tensors for NN training
                    X_train = torch.from_numpy(X_train).float()
                    X_test = torch.from_numpy(X_test).float()
                    y_train = torch.from_numpy(y_train).long()
                    y_test = torch.from_numpy(y_test).long()

                    classes_all = [0,1,2,3] 
                    
                    #initilize model
                    model = Model_current(chn_inp = X_train.size()[-2], len_inp = X_train.size()[-1], nf = int(config['nf']), ks = int(config['ks']) , 
                                  stride = int(config['stride']), act_f = config['act_fun'], nfc = int(config['nfc']))

                    #set save_path to None if we dont want to save the nn weights
                    save_path = None
                    if to_save == 1:
                        save_path = file_root_save + '\\ANN\\model_config_'+ str(make_config_str(config)) + '_' + 'n_inits_' + str(n_inits)+ '_fold_' + str(fold) + '.pt'
                        #load in pre_trainied orignial model parameters
                        model.load_state_dict(torch.load(save_path))
                        
                        #copy original (300 epochs) version of model parameters
                        copy_path  = file_root_save + '\\ANN\\original_model_config_'+ str(make_config_str(config)) + '_' + 'n_inits_' + str(n_inits)+ '_fold_' + str(fold) + '.pt'
                        torch.save(torch.load(save_path), copy_path)
                        #copy original (300 epochs) version of training and testing accuracies
                        orig_Train_acc_c = pickle.load(open(file_root_save + '\\ANN\\Train_acc_c_config_'+ make_config_str(config) + '_' + str(n_inits)+ '_fold_' + str(fold) + ".pickle", "rb" ) )
                        orig_Test_acc_c = pickle.load(open(file_root_save + '\\ANN\\Test_acc_c_config_'+ make_config_str(config) + '_' + str(n_inits)+ '_fold_' + str(fold) + ".pickle", "rb" ) )
                        pickle.dump( orig_Train_acc_c, open(file_root_save + '\\ANN\\original_Train_acc_c_config_'+ make_config_str(config) + '_' + str(n_inits)+ '_fold_' + str(fold) + ".pickle", "wb" ) )
                        pickle.dump( orig_Test_acc_c, open(file_root_save + '\\ANN\\original_Test_acc_c_config_'+ make_config_str(config) + '_' + str(n_inits)+ '_fold_' + str(fold) + ".pickle", "wb" ) )

                    
                    Loss = nn.CrossEntropyLoss()

                    if config['optim'] == 'SGD':
                        lr = 0.2
                        optimizer = optim.SGD(model.parameters(), lr = lr)
                        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor= 0.5, min_lr=0.001, cooldown = 5)

                    elif 'ADAM_' in config['optim']:
                        lr = float(config['optim'].split('_')[1])
                        optimizer = optim.Adam(model.parameters(), lr = lr)
                        scheduler = None

                    #Train and test NN and save the best weights
                    Train_acc_c, Test_acc_c, Train_loss, Test_loss, best_epoch, best_eval_acc, best_eval_acc_c = train_test_NN(
                        save_path, X_train, y_train.long(), X_test, y_test.long(), 
                                      model, Loss, optimizer, scheduler, num_epoch, batch_size, classes_all)
                    
                    if to_save == 1:
                        pickle.dump( Train_acc_c, open(file_root_save + '\\ANN\\Train_acc_c_config_'+ make_config_str(config) + '_' + str(n_inits)+ '_fold_' + str(fold) + ".pickle", "wb" ) )
                        pickle.dump( Test_acc_c, open(file_root_save +  '\\ANN\\Test_acc_c_config_'+ make_config_str(config) + '_' + str(n_inits)+  '_fold_' + str(fold) + ".pickle", "wb" ) )
                    
                    plot= 0
                    if n_inits == 0 and plot == 1:
                        plt.plot(np.average(Train_acc_c, axis = 1), 'b')
                        plt.plot(np.average(Test_acc_c, axis = 1), 'r')
                        plt.show()
                        
                    #calc accuracy
                    fold_ave = np.average(best_eval_acc_c)
                    #print ('fold', fold, ' fold acc ', fold_ave)
                    ACC.append(fold_ave)
                    
            if to_save == 1:
                orig_ACC = pickle.load(open(file_root_save + '\\ANN\\ACC_' + make_config_str(config) + '.pickle', 'rb'))
                pickle.dump(orig_ACC, open(file_root_save + '\\ANN\\original_ACC_' + make_config_str(config) + '.pickle', 'wb'))
                
                pickle.dump(ACC, open(file_root_save + '\\ANN\\ACC_' + make_config_str(config) + '.pickle', 'wb'))


A01T
1


D:\Diamond\code\thesis_funcs_19_03.py:1334: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nnF.softmax(self.fc3(x))


0.7
0.5
0.3
0.1
A02T
1
0.7
0.5
0.3
0.1
A03T
1
0.7
0.5
0.3
0.1
A04T
1
0.7
0.5
0.3
0.1
A05T
1
0.7
0.5
0.3
0.1
A06T
1
0.7
0.5
0.3
0.1
A07T
1
0.7
0.5
0.3
0.1
A08T
1
0.7
0.5
0.3
0.1
A09T
1
0.7
0.5
0.3
0.1


In [35]:
file_root_save

'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\CURRENT\\tl_comp_csp_mi\\A09\\4s\\pt_10'

In [11]:
print (config)

OrderedDict([('act_fun', 'leaky_relu'), ('nf', '32'), ('ks', '4'), ('stride', '2'), ('nfc', '60'), ('optim', 'ADAM_0.001'), ('_lambda', '0')])


In [7]:
hp_names

['act_fun', 'nf', 'ks', 'stride', 'nfc', 'optim', '_lambda']

In [33]:
save_root

'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\CURRENT\\tl_comp_csp_mi\\'